In [1]:
import json, os

In [5]:
print('Current working directory: ' + os.getcwd())
os.chdir('../downloads')
print('New working directory: ' + os.getcwd())

Current working directory: c:\Users\Boonchong\Desktop\defense_of_the_ancient\notebooks
New working directory: c:\Users\Boonchong\Desktop\defense_of_the_ancient\downloads


In [10]:
db = []
for f in os.listdir(os.getcwd()+ os.sep + '230821'):
    file = open(os.getcwd() + os.sep + '230821' + os.sep + f)
    data = json.load(file)
    file.close()

In [12]:
data.keys()

dict_keys(['match_id', 'barracks_status_dire', 'barracks_status_radiant', 'chat', 'cluster', 'cosmetics', 'dire_score', 'dire_team_id', 'draft_timings', 'duration', 'engine', 'first_blood_time', 'game_mode', 'human_players', 'leagueid', 'lobby_type', 'match_seq_num', 'negative_votes', 'objectives', 'picks_bans', 'positive_votes', 'radiant_gold_adv', 'radiant_score', 'radiant_team_id', 'radiant_win', 'radiant_xp_adv', 'skill', 'start_time', 'teamfights', 'tower_status_dire', 'tower_status_radiant', 'version', 'replay_salt', 'series_id', 'series_type', 'players', 'patch', 'region', 'replay_url'])